# Objetivo del notebook

- Limpieza y transformación de datos.
- Preparar df_fleca con datos de hostelería y venta de pan para el EDA y forecasting.

# Cargar datos raw

In [165]:
# Cargar el archivo parquet en un DataFrame llamado df_fleca
# Usamos pyarrow.parquet para poder acceder al schema y tipos de columnas del archivo parquet,
# lo que permite detectar y convertir tipos especiales como 'dbdate' antes de cargar a pandas.
import pandas as pd
import pyarrow.parquet as pq

# Importar path centralizado
try:
    from src.paths import RAW_BQ_PARQUET
except ImportError:
    import sys
    sys.path.append('..')
    from src.paths import RAW_BQ_PARQUET

parquet_file = str(RAW_BQ_PARQUET)
table = pq.read_table(parquet_file)
schema = table.schema

# Identificar columnas con tipo 'dbdate'
dbdate_cols = [field.name for field in schema if str(field.type) == 'dbdate']

# Convertir columnas 'dbdate' a string en el objeto Table de pyarrow
for col in dbdate_cols:
    table = table.set_column(table.schema.get_field_index(col), col, table.column(col).cast('string'))

# Convertir a pandas desactivando la metadata de pandas
df_fleca = table.to_pandas(ignore_metadata=True)

df_fleca.head()

,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total
0,2024-01-10,T/112704,S6,2 CHURROS HORNO,BOLLERIA,1.0,1.18,10.0,1.3
1,2024-01-10,T/112704,S6,2 CHURROS HORNO,BOLLERIA,1.0,1.18,10.0,1.3
2,2024-01-09,T/112555,LL5,7 CEREALES BUT. BLANCA,BOCADILLOS,1.0,2.18,10.0,2.4
3,2024-01-08,T/112397,B4,7 CEREALES BUT. BLANCA,BOCADILLOS,1.0,2.18,10.0,2.4
4,2024-01-02,T/111637,S4,7 CEREALES BUT. BLANCA,BOCADILLOS,1.0,2.18,10.0,2.4


# Revisión de integridad temporal

In [166]:
# Verificar que todas las fechas diarias entre enero 2023 y junio 2025 están presentes en df_fleca.
#Detectar huecos temporales

import numpy as np

# Crear el rango completo de fechas diarias
fechas_completas = pd.date_range(start='2023-01-01', end='2025-06-30', freq='D')

# Convertir la columna 'fecha' a datetime si no lo está
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'])

# Obtener las fechas únicas presentes en el DataFrame
fechas_presentes = pd.Series(df_fleca['fecha'].unique())

# Verificar qué fechas faltan
fechas_faltantes = np.setdiff1d(fechas_completas, fechas_presentes)

print(f"Total de fechas faltantes: {len(fechas_faltantes)}")
print(fechas_faltantes)

Total de fechas faltantes: 5
['2023-01-01T00:00:00.000000000' '2023-12-25T00:00:00.000000000'
 '2024-01-01T00:00:00.000000000' '2024-12-25T00:00:00.000000000'
 '2025-01-01T00:00:00.000000000']


In [167]:
# Valores nulos
print(df_fleca.isnull().sum())


fecha                0
n_factura            0
zona_de_venta        0
producto             0
familia              1
cantidad             0
base_imponible    2248
tipo_IVA          2247
total             2248
dtype: int64


## Tratamiento de datos faltantes

## Tratamiento de días faltantes


In [168]:
# Crear un DataFrame con las fechas faltantes
df_fechas_faltantes = pd.DataFrame({'fecha': fechas_faltantes})

# Añadir columnas con valores por defecto
df_fechas_faltantes['n_factura'] = 'cerrado'
df_fechas_faltantes['zona_de_venta'] = 'cerrado'
df_fechas_faltantes['producto'] = 'cerrado'
df_fechas_faltantes['familia'] = 'cerrado'
df_fechas_faltantes['cantidad'] = 0.0
df_fechas_faltantes['base_imponible'] = 0.0
df_fechas_faltantes['tipo_IVA'] = 0.0
df_fechas_faltantes['total'] = 0.0
df_fechas_faltantes['is_closed'] = 1

# Añadir la columna is_closed al df_fleca, inicializada en 0
df_fleca['is_closed'] = 0

# Unir ambos DataFrames
df_fleca = pd.concat([df_fleca, df_fechas_faltantes], ignore_index=True)

# Eliminar posibles duplicados de fechas faltantes antes de concatenar
df_fechas_faltantes = df_fechas_faltantes[~df_fechas_faltantes['fecha'].isin(df_fleca['fecha'])]

# Ordenar por fecha
df_fleca = df_fleca.sort_values('fecha').reset_index(drop=True)
# Verificar duplicados de la columna 'fecha' y mostrar cuántos hay por fecha
# Antes de crear df_fechas_faltantes, asegúrate de que solo se agregue una fila por cada fecha faltante
# Elimina del DataFrame principal las fechas faltantes que ya están presentes (por si hay duplicados)
fechas_faltantes_unicas = np.setdiff1d(fechas_faltantes, df_fleca['fecha'].unique())
df_fechas_faltantes = pd.DataFrame({'fecha': fechas_faltantes_unicas})


df_fleca.head(30)

,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total,is_closed
0,2023-01-01,cerrado,cerrado,cerrado,cerrado,0.00,0.00,0.0,0.00,1
1,2023-01-02,T/044122,S4,CACAOLAT,BEBIDAS,1.00,1.91,10.0,2.10,0
2,2023-01-02,T/044001,LL5,PETIT ARTESANO (Kg),BOLLERIA,0.08,1.35,10.0,1.48,0
3,2023-01-02,T/044029,LL5,NAPO CREMA,BOLLERIA,1.00,1.55,10.0,1.70,0
4,2023-01-02,T/044057,T3,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0
5,2023-01-02,T/043965,LL5,PETIT ARTESANO (Kg),BOLLERIA,0.08,1.26,10.0,1.39,0
6,2023-01-02,T/044051,LL5,PETIT ARTESANO (Kg),BOLLERIA,0.08,1.26,10.0,1.39,0
7,2023-01-02,T/044083,S2,HUEVOS KINDER,VARIOS,1.00,1.59,10.0,1.75,0
8,2023-01-02,T/044046,T1,CAFE LLET CIVADA,CAFES,1.00,0.09,10.0,0.10,0
9,2023-01-02,T/044055,T3,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0


In [169]:
# Comprobar que se han imputado bien y no faltan fechas en el rango completo
fechas_unicas = pd.Series(df_fleca['fecha'].unique())
fechas_faltantes_final = np.setdiff1d(fechas_completas, fechas_unicas)

print(f"Fechas faltantes en el rango 2023-01-01 a 2025-06-30: {len(fechas_faltantes_final)}")
if len(fechas_faltantes_final) == 0:
    print("No faltan fechas en el rango especificado.")
else:
    print(fechas_faltantes_final)

Fechas faltantes en el rango 2023-01-01 a 2025-06-30: 0
No faltan fechas en el rango especificado.


## Tratamiento de Valores Nulos en Otras Columnas

In [170]:
# Agrupar por mes y por familia (categoría) los valores nulos en las columnas relevantes
df_fleca['mes'] = df_fleca['fecha'].dt.to_period('M')

# Seleccionar columnas con posibles nulos
cols_nulos = ['base_imponible', 'tipo_IVA', 'total']

# Calcular valores nulos por mes y familia
valores_nulos_mes_familia = (
    df_fleca[df_fleca[cols_nulos].isnull().any(axis=1)]
    .groupby(['mes', 'familia'])[cols_nulos]
    .apply(lambda x: x.isnull().sum())
)

print(valores_nulos_mes_familia)

                          base_imponible  tipo_IVA  total
mes     familia                                          
2023-11 AÑADIDOS                      11        11     11
        BEBIDA                        98        98     98
        BEBIDAS                       87        87     87
        BOCADILLOS                   329       329    329
        BOLLERIA                     426       426    426
        CAFES                        872       872    872
        CERVEZAS                      24        24     24
        LICORES                        8         8      8
        PAN                          260       260    260
        PASTELERIA                    55        55     55
        TES & INFUSIONES              44        44     44
        TOSTADAS                      21        21     21
        VARIOS                        12        12     12
2025-06 BOLLERIA                       1         0      1


In [171]:
# Imputar valores nulos en columnas especificadas usando medias por familia
# Versión optimizada para evitar bucles lentos

# Inicializar columna is_imputed si no existe
if 'is_imputed' not in df_fleca.columns:
    df_fleca['is_imputed'] = 0

# Crear máscaras una sola vez
mask_nov_2023 = df_fleca['mes'] == pd.Period('2023-11', 'M')
mask_oct_dic_2023 = df_fleca['mes'].isin([pd.Period('2023-10', 'M'), pd.Period('2023-12', 'M')])
mask_2023 = df_fleca['mes'].dt.year == 2023

# Calcular medias por familia una sola vez
medias_oct_dic = df_fleca[mask_oct_dic_2023].groupby('familia')[cols_nulos].mean()
medias_anuales = df_fleca[mask_2023].groupby('familia')[cols_nulos].mean()

# Procesar cada familia
familias = df_fleca['familia'].unique()
print(f"Procesando {len(familias)} familias...")

for i, familia in enumerate(familias):
    if i % 10 == 0:  # Mostrar progreso cada 10 familias
        print(f"Procesando familia {i+1}/{len(familias)}")
    
    # Máscara para noviembre 2023 de esta familia
    mask_familia_nov = mask_nov_2023 & (df_fleca['familia'] == familia)
    
    # Inicializar is_imputed = 0 para esta familia en noviembre
    df_fleca.loc[mask_familia_nov, 'is_imputed'] = 0
    
    # Procesar cada columna
    for columna in cols_nulos:
        # Máscara para nulos en noviembre de esta familia y columna
        mask_nulos = mask_familia_nov & df_fleca[columna].isnull()
        
        if not mask_nulos.any():
            continue  # No hay nulos que imputar en esta columna para esta familia
        
        # Intentar con media octubre-diciembre
        if familia in medias_oct_dic.index and not pd.isna(medias_oct_dic.loc[familia, columna]):
            valor_imputacion = medias_oct_dic.loc[familia, columna]
            df_fleca.loc[mask_nulos, columna] = valor_imputacion
            df_fleca.loc[mask_nulos, 'is_imputed'] = 1
        # Si no, usar media anual
        elif familia in medias_anuales.index and not pd.isna(medias_anuales.loc[familia, columna]):
            valor_imputacion = medias_anuales.loc[familia, columna]
            df_fleca.loc[mask_nulos, columna] = valor_imputacion
            df_fleca.loc[mask_nulos, 'is_imputed'] = 1
        # Si tampoco hay media anual, is_imputed ya está en 0

print("Imputación completada.")

Procesando 23 familias...


Procesando familia 1/23
Procesando familia 11/23
Procesando familia 21/23
Imputación completada.


In [172]:
# Comprobar si se han imputado correctamente los valores nulos
# Mostrar registros donde is_imputed == 1
print(df_fleca[df_fleca['is_imputed'] == 1][['fecha', 'familia', 'base_imponible', 'tipo_IVA', 'total','is_imputed']])

            fecha     familia  base_imponible   tipo_IVA     total  is_imputed
131579 2023-11-03    BOLLERIA        1.706340  10.000000  1.876947           1
131583 2023-11-03    BOLLERIA        1.706340  10.000000  1.876947           1
131585 2023-11-03         PAN        1.366413   0.526316  1.374109           1
131587 2023-11-03       CAFES        1.593709  10.000000  1.754567           1
131588 2023-11-03    BOLLERIA        1.706340  10.000000  1.876947           1
...           ...         ...             ...        ...       ...         ...
134121 2023-11-10  BOCADILLOS        2.680573  10.000000  2.948817           1
134122 2023-11-10      BEBIDA        1.826201  10.000000  2.009752           1
134123 2023-11-10       CAFES        1.593709  10.000000  1.754567           1
134124 2023-11-10    BOLLERIA        1.706340  10.000000  1.876947           1
134125 2023-11-10  BOCADILLOS        2.680573  10.000000  2.948817           1

[2247 rows x 6 columns]


In [173]:
# Agrupar por mes y por familia (categoría) los valores nulos en las columnas relevantes
df_fleca['mes'] = df_fleca['fecha'].dt.to_period('M')

# Seleccionar columnas con posibles nulos
cols_nulos = ['base_imponible', 'tipo_IVA', 'total']

# Calcular valores nulos por mes y familia
valores_nulos_mes_familia = (
    df_fleca[df_fleca[cols_nulos].isnull().any(axis=1)]
    .groupby(['mes', 'familia'])[cols_nulos]
    .apply(lambda x: x.isnull().sum())
)

print(valores_nulos_mes_familia)

                  base_imponible  tipo_IVA  total
mes     familia                                  
2025-06 BOLLERIA               1         0      1


In [174]:
# Eliminar los valores nulos
df_fleca = df_fleca.dropna(subset=cols_nulos)

In [175]:
df_fleca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 337357 entries, 0 to 337357
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           337357 non-null  datetime64[ns]
 1   n_factura       337357 non-null  object        
 2   zona_de_venta   337357 non-null  object        
 3   producto        337357 non-null  object        
 4   familia         337356 non-null  object        
 5   cantidad        337357 non-null  float64       
 6   base_imponible  337357 non-null  float64       
 7   tipo_IVA        337357 non-null  float64       
 8   total           337357 non-null  float64       
 9   is_closed       337357 non-null  int64         
 10  mes             337357 non-null  period[M]     
 11  is_imputed      337357 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(4), period[M](1)
memory usage: 33.5+ MB


In [176]:
df_fleca.head()

,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total,is_closed,mes,is_imputed
0,2023-01-01,cerrado,cerrado,cerrado,cerrado,0.00,0.00,0.0,0.00,1,2023-01,0
1,2023-01-02,T/044122,S4,CACAOLAT,BEBIDAS,1.00,1.91,10.0,2.10,0,2023-01,0
2,2023-01-02,T/044001,LL5,PETIT ARTESANO (Kg),BOLLERIA,0.08,1.35,10.0,1.48,0,2023-01,0
3,2023-01-02,T/044029,LL5,NAPO CREMA,BOLLERIA,1.00,1.55,10.0,1.70,0,2023-01,0
4,2023-01-02,T/044057,T3,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0,2023-01,0


# Homogeneización

In [177]:
# Homogeneización de la columna 'fecha' 
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'])

print(df_fleca.dtypes)


fecha             datetime64[ns]
n_factura                 object
zona_de_venta             object
producto                  object
familia                   object
cantidad                 float64
base_imponible           float64
tipo_IVA                 float64
total                    float64
is_closed                  int64
mes                    period[M]
is_imputed                 int64
dtype: object


In [178]:
# Convertir la columna 'fecha' a datetime antes de eliminar horas, minutos y segundos
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'])

# Verificar la consistencia de la columna familia
print(df_fleca['familia'].unique())

['cerrado' 'BEBIDAS' 'BOLLERIA' 'CAFES' 'VARIOS' 'PASTELERIA' 'BOCADILLOS'
 'PAN' 'AÑADIDOS' 'BEBIDA' 'TES & INFUSIONES' 'LICORES' 'CERVEZAS'
 'TOSTADAS' 'LLET' None 'AMERI' 'º' 'PETI' '2' '7' 'PETIT XOCO' 'AIGU']


In [179]:
# Identificar los meses cercanos a noviembre 2023
meses_cercanos = ["2023-10", "2023-12"]

# Convertir a formato Periodo (mes-año) para facilitar la agrupación
df_fleca['mes'] = df_fleca['fecha'].dt.to_period("M")

# Imputar valores nulos en 'base_imponible', 'tipo_IVA' y 'total' usando la media de la categoría en meses cercanos
for columna in ['base_imponible', 'tipo_IVA', 'total']:
    for categoria in df_fleca['familia'].unique():
        for mes in meses_cercanos:
            # Filtrar la media de la categoría en los meses cercanos
            media_categoria = df_fleca[(df_fleca['familia'] == categoria) & (df_fleca['mes'] == mes)][columna].mean()
            
            # Verificar si la media es válida
            if not pd.isna(media_categoria):
                # Rellenar valores nulos solo en noviembre 2023 de esa categoría
                df_fleca.loc[
                    (df_fleca['familia'] == categoria) &
                    (df_fleca['mes'] == "2023-11") &
                    (df_fleca[columna].isnull()),
                    columna
                ] = media_categoria

# Si sigue habiendo valores nulos, usar la media anual de la categoría en 2023
for columna in ['base_imponible', 'tipo_IVA', 'total']:
    for categoria in df_fleca['familia'].unique():
        media_anual = df_fleca[(df_fleca['familia'] == categoria) & (df_fleca['mes'].dt.year == 2023)][columna].mean()
        
        # Verificar si la media anual es válida
        if not pd.isna(media_anual):
            df_fleca.loc[
                (df_fleca['familia'] == categoria) &
                (df_fleca['mes'] == "2023-11") &
                (df_fleca[columna].isnull()),
                columna
            ] = media_anual

# Verificar si quedan valores nulos
print("Valores nulos después de la imputación:")
print(df_fleca.isnull().sum())

Valores nulos después de la imputación:
fecha             0
n_factura         0
zona_de_venta     0
producto          0
familia           1
cantidad          0
base_imponible    0
tipo_IVA          0
total             0
is_closed         0
mes               0
is_imputed        0
dtype: int64


In [180]:
# Homogeneizar la categoría 'Bebidas'
# Mostrar el total de filas de la familia "BEBIDAS" Y "BEBIDA" por separado

print(df_fleca[df_fleca['familia'] == 'BEBIDAS'].shape[0])
print(df_fleca[df_fleca['familia'] == 'BEBIDA'].shape[0])   


14339
18350


In [181]:
# Cambiar el nombre de las observaciones de la familia "BEBIDA" a "BEBIDAS"
df_fleca.loc[df_fleca['familia'] == 'BEBIDA', 'familia'] = 'BEBIDAS'
# Verificar que se han cambiado los nombres
print(df_fleca['familia'].unique())

['cerrado' 'BEBIDAS' 'BOLLERIA' 'CAFES' 'VARIOS' 'PASTELERIA' 'BOCADILLOS'
 'PAN' 'AÑADIDOS' 'TES & INFUSIONES' 'LICORES' 'CERVEZAS' 'TOSTADAS'
 'LLET' None 'AMERI' 'º' 'PETI' '2' '7' 'PETIT XOCO' 'AIGU']


In [182]:
# Contar los registros de la familia '7' 'AIGU'  'AMERI' 'LLET' 'º' None 'PETI' 'PETIT XOCO' '2']
print(df_fleca[df_fleca['familia'] == '7'].shape[0])
print(df_fleca[df_fleca['familia'] == 'AIGU'].shape[0])
print(df_fleca[df_fleca['familia'] == 'AMERI'].shape[0])  
print(df_fleca[df_fleca['familia'] == 'LLET'].shape[0])
print(df_fleca[df_fleca['familia'] == 'º'].shape[0])
print(df_fleca[df_fleca['familia'] == None].shape[0])
print(df_fleca[df_fleca['familia'] == 'PETI'].shape[0])
print(df_fleca[df_fleca['familia'] == 'PETIT XOCO'].shape[0])
print(df_fleca[df_fleca['familia'] == '2'].shape[0])          

1
1
1
2
1
0
1
1
1


In [183]:
#Eliminar los registros de la familia '7' 'AIGU'  'AMERI' 'LLET' 'º' None 'PETI' 'PETIT XOCO' '2']
df_fleca = df_fleca[~df_fleca['familia'].isin(['7', 'AIGU', 'AMERI', 'LLET', 'º', None, 'PETI', 'PETIT XOCO', '2'])]
# Verificar que se han eliminado los registros
print(df_fleca['familia'].unique())

['cerrado' 'BEBIDAS' 'BOLLERIA' 'CAFES' 'VARIOS' 'PASTELERIA' 'BOCADILLOS'
 'PAN' 'AÑADIDOS' 'TES & INFUSIONES' 'LICORES' 'CERVEZAS' 'TOSTADAS']


In [184]:
# Normalizar la columna 'producto': convertir a mayúsculas y eliminar espacios extra
df_fleca['producto'] = df_fleca['producto'].str.upper().str.strip()

In [185]:
df_fleca.head()

,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total,is_closed,mes,is_imputed
0,2023-01-01,cerrado,cerrado,CERRADO,cerrado,0.00,0.00,0.0,0.00,1,2023-01,0
1,2023-01-02,T/044122,S4,CACAOLAT,BEBIDAS,1.00,1.91,10.0,2.10,0,2023-01,0
2,2023-01-02,T/044001,LL5,PETIT ARTESANO (KG),BOLLERIA,0.08,1.35,10.0,1.48,0,2023-01,0
3,2023-01-02,T/044029,LL5,NAPO CREMA,BOLLERIA,1.00,1.55,10.0,1.70,0,2023-01,0
4,2023-01-02,T/044057,T3,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0,2023-01,0


# Generación de variables exógenas

In [186]:
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'])

df_fleca['month'] = df_fleca['fecha'].dt.month
# El método .dt.dayofweek ya asigna 0 para lunes y 6 para domingo, por lo que no es necesario modificar nada.
df_fleca['day_of_week'] = df_fleca['fecha'].dt.dayofweek
df_fleca['is_weekend'] = df_fleca['day_of_week'].isin([5, 6]).astype(int)
df_fleca['is_summer_peak'] = df_fleca['month'].isin([7, 8]).astype(int)

df_fleca.head(500)


,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total,is_closed,mes,is_imputed,month,day_of_week,is_weekend,is_summer_peak
0,2023-01-01,cerrado,cerrado,CERRADO,cerrado,0.00,0.00,0.0,0.00,1,2023-01,0,1,6,1,0
1,2023-01-02,T/044122,S4,CACAOLAT,BEBIDAS,1.00,1.91,10.0,2.10,0,2023-01,0,1,0,0,0
2,2023-01-02,T/044001,LL5,PETIT ARTESANO (KG),BOLLERIA,0.08,1.35,10.0,1.48,0,2023-01,0,1,0,0,0
3,2023-01-02,T/044029,LL5,NAPO CREMA,BOLLERIA,1.00,1.55,10.0,1.70,0,2023-01,0,1,0,0,0
4,2023-01-02,T/044057,T3,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0,2023-01,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023-01-03,T/044197,S6,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0,2023-01,0,1,1,0,0
496,2023-01-03,T/044235,T2,TALLAT,CAFES,1.00,1.27,10.0,1.40,0,2023-01,0,1,1,0,0
497,2023-01-03,T/044243,S8,CAFE EXPRESO,CAFES,1.00,1.18,10.0,1.30,0,2023-01,0,1,1,0,0
498,2023-01-03,T/044209,S4,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0,2023-01,0,1,1,0,0


In [187]:
import pandas as pd

# Partimos de df_fleca ya corregido
df = df_fleca.copy()
df['fecha'] = pd.to_datetime(df['fecha'])

# Definir los rangos de Semana Santa por año (ajusta según tu calendario)
easter_ranges = {
    2023: pd.date_range('2023-04-03', '2023-04-09'),  # Lunes Santo a Domingo de Resurrección
    2024: pd.date_range('2024-03-25', '2024-03-31'),
    2025: pd.date_range('2025-04-14', '2025-04-20'),
}

# Crear la columna is_easter con 0 por defecto
df['is_easter'] = 0
for year, dates in easter_ranges.items():
    df.loc[df['fecha'].dt.date.isin(dates.date), 'is_easter'] = 1

# Filtramos solo los días marcados como is_easter
df_e = df.loc[df['is_easter'] == 1].copy()

# 1) Número de fechas únicas marcadas por año
counts = df_e.groupby(df_e['fecha'].dt.year)['fecha'].apply(lambda x: x.dt.date.nunique())
print("Días únicos marcados como Semana Santa por año:")
print(counts)

# 2) Listado de las fechas marcadas por año
print("\nFechas marcadas como Semana Santa por año:")
for year, group in df_e.groupby(df_e['fecha'].dt.year):
    unique_days = sorted(group['fecha'].dt.date.unique())
    print(f"{year}: {unique_days}")

Días únicos marcados como Semana Santa por año:
fecha
2023    7
2024    7
2025    7
Name: fecha, dtype: int64

Fechas marcadas como Semana Santa por año:
2023: [datetime.date(2023, 4, 3), datetime.date(2023, 4, 4), datetime.date(2023, 4, 5), datetime.date(2023, 4, 6), datetime.date(2023, 4, 7), datetime.date(2023, 4, 8), datetime.date(2023, 4, 9)]
2024: [datetime.date(2024, 3, 25), datetime.date(2024, 3, 26), datetime.date(2024, 3, 27), datetime.date(2024, 3, 28), datetime.date(2024, 3, 29), datetime.date(2024, 3, 30), datetime.date(2024, 3, 31)]
2025: [datetime.date(2025, 4, 14), datetime.date(2025, 4, 15), datetime.date(2025, 4, 16), datetime.date(2025, 4, 17), datetime.date(2025, 4, 18), datetime.date(2025, 4, 19), datetime.date(2025, 4, 20)]


In [188]:
# Convertir la columna 'fecha' a datetime si no lo está
df_fleca['fecha'] = pd.to_datetime(df_fleca['fecha'])

df_fleca.head()

,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total,is_closed,mes,is_imputed,month,day_of_week,is_weekend,is_summer_peak
0,2023-01-01,cerrado,cerrado,CERRADO,cerrado,0.00,0.00,0.0,0.00,1,2023-01,0,1,6,1,0
1,2023-01-02,T/044122,S4,CACAOLAT,BEBIDAS,1.00,1.91,10.0,2.10,0,2023-01,0,1,0,0,0
2,2023-01-02,T/044001,LL5,PETIT ARTESANO (KG),BOLLERIA,0.08,1.35,10.0,1.48,0,2023-01,0,1,0,0,0
3,2023-01-02,T/044029,LL5,NAPO CREMA,BOLLERIA,1.00,1.55,10.0,1.70,0,2023-01,0,1,0,0,0
4,2023-01-02,T/044057,T3,CAFE LLET,CAFES,1.00,1.45,10.0,1.60,0,2023-01,0,1,0,0,0


# Validación de la continuidad
Verificamos que no queden nulos en ningún campo

In [189]:
print(df_fleca.isnull().sum())


fecha             0
n_factura         0
zona_de_venta     0
producto          0
familia           0
cantidad          0
base_imponible    0
tipo_IVA          0
total             0
is_closed         0
mes               0
is_imputed        0
month             0
day_of_week       0
is_weekend        0
is_summer_peak    0
dtype: int64


In [190]:
df_fleca.info()


<class 'pandas.core.frame.DataFrame'>
Index: 337347 entries, 0 to 337357
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   fecha           337347 non-null  datetime64[ns]
 1   n_factura       337347 non-null  object        
 2   zona_de_venta   337347 non-null  object        
 3   producto        337347 non-null  object        
 4   familia         337347 non-null  object        
 5   cantidad        337347 non-null  float64       
 6   base_imponible  337347 non-null  float64       
 7   tipo_IVA        337347 non-null  float64       
 8   total           337347 non-null  float64       
 9   is_closed       337347 non-null  int64         
 10  mes             337347 non-null  period[M]     
 11  is_imputed      337347 non-null  int64         
 12  month           337347 non-null  int32         
 13  day_of_week     337347 non-null  int32         
 14  is_weekend      337347 non-null  int64   

# Preparación de la granularidad base

## Agregación diaria

In [191]:
import pandas as pd

# Agregación por fecha y familia

# Agregamos la base imponible y el total por fecha y familia
df_fleca_agg = df_fleca.groupby(['fecha', 'familia']).agg({
    'base_imponible': 'sum',
    'total': 'sum'
}).reset_index()

# Convertir la columna 'fecha' a datetime
df_fleca_agg['fecha'] = pd.to_datetime(df_fleca_agg['fecha'])

# Paso 3: Enriquecimiento con variables de calendario y exógenas

# Variables de calendario
df_fleca_agg['day_of_week'] = df_fleca_agg['fecha'].dt.dayofweek  # 0=Lunes, 6=Domingo
df_fleca_agg['month'] = df_fleca_agg['fecha'].dt.month
df_fleca_agg['week'] = df_fleca_agg['fecha'].dt.isocalendar().week
df_fleca_agg['year'] = df_fleca_agg['fecha'].dt.year
df_fleca_agg['is_weekend'] = df_fleca_agg['day_of_week'].isin([5,6]).astype(int)

# Variables exógenas
df_fleca_agg['is_summer_peak'] = df_fleca_agg['month'].isin([7,8]).astype(int)

df_fleca_agg['is_easter'] = df_fleca_agg['fecha'].apply(
    lambda x: 1 if (x.month == 3 and x.day >= 24) or (x.month == 4 and x.day <= 10) else 0
)

# Mostramos un resumen rápido para validar la estructura
df_fleca_agg.head()




,fecha,familia,base_imponible,total,day_of_week,month,week,year,is_weekend,is_summer_peak,is_easter
0,2023-01-01,cerrado,0.00,0.00,6,1,52,2023,1,0,0
1,2023-01-02,AÑADIDOS,2.18,2.40,0,1,1,2023,0,0,0
2,2023-01-02,BEBIDAS,70.75,77.80,0,1,1,2023,0,0,0
3,2023-01-02,BOCADILLOS,134.41,147.75,0,1,1,2023,0,0,0
4,2023-01-02,BOLLERIA,101.96,112.16,0,1,1,2023,0,0,0


In [192]:
# Guardar el DataFrame transformado en la carpeta interim usando path centralizado
from src.paths import VALIDATED_RANGE_FECHA_FAMILIA
df_fleca_agg.to_parquet(str(VALIDATED_RANGE_FECHA_FAMILIA), index=False)

## Agregación semanal

In [195]:
import pandas as pd
import sys
sys.path.append('..')  # Para asegurar que src esté en el path
from src.paths import VALIDATED_RANGE_SEMANAL_FAMILIA

# --- Agregación Semanal Dinámica ---
def aggregate_weekly(df_daily: pd.DataFrame) -> pd.DataFrame:
    """Genera vista semanal a partir de df_daily"""
    df_weekly = (
        df_daily
        .set_index('fecha')
        .groupby('familia')[['base_imponible','total','is_easter','is_summer_peak']]
        .resample('W')
        .sum()
        .reset_index()
    )
    return df_weekly

# Generar y guardar df_weekly
df_weekly = aggregate_weekly(df_fleca_agg)
df_weekly.to_parquet(str(VALIDATED_RANGE_SEMANAL_FAMILIA), index=False)
print(f"Guardado df_weekly en: {VALIDATED_RANGE_SEMANAL_FAMILIA}")

ImportError: cannot import name 'VALIDATED_RANGE_SEMANAL_FAMILIA' from 'src.paths' (C:\Workspace\mlops_fleca_project\src\paths.py)

In [ ]:
df_weekly.tail(500)

,familia,fecha,base_imponible,total,is_easter,is_summer_peak
1178,PASTELERIA,2025-07-06,39.58,43.55,0,0
1179,TES & INFUSIONES,2023-01-08,92.25,101.60,0,0
1180,TES & INFUSIONES,2023-01-15,107.91,118.90,0,0
1181,TES & INFUSIONES,2023-01-22,68.82,75.85,0,0
1182,TES & INFUSIONES,2023-01-29,105.95,116.75,0,0
...,...,...,...,...,...,...
1673,cerrado,2024-12-08,0.00,0.00,0,0
1674,cerrado,2024-12-15,0.00,0.00,0,0
1675,cerrado,2024-12-22,0.00,0.00,0,0
1676,cerrado,2024-12-29,0.00,0.00,0,0


## Agregación mensual

In [ ]:
from src.paths import VALIDATED_RANGE_MONTHLY_FAMILIA

# --- Agregación Mensual Dinámica ---
def aggregate_monthly(df_daily: pd.DataFrame) -> pd.DataFrame:
    """Genera vista mensual a partir de df_daily, asegurando continuidad y consistencia."""
    meses = pd.date_range(df_daily['fecha'].min(), df_daily['fecha'].max(), freq='M')
    familias = df_daily['familia'].unique()
    comb = pd.MultiIndex.from_product([meses, familias], names=['fecha','familia']).to_frame(index=False)
    
    # Agregar mensual
    df_monthly = (
        df_daily
        .set_index('fecha')
        .groupby('familia')[['base_imponible','total','is_easter','is_summer_peak']]
        .resample('M')
        .sum()
        .reset_index()
    )
    # Unir para asegurar continuidad
    df_monthly_full = (
        comb
        .merge(df_monthly, on=['fecha','familia'], how='left')
        .fillna({
            'base_imponible': 0,
            'total': 0,
            'is_easter': 0,
            'is_summer_peak': 0
        })
        .sort_values(['familia','fecha'])
    )
    return df_monthly_full

# Generar y guardar df_monthly
df_monthly = aggregate_monthly(df_fleca_agg)
df_monthly.to_parquet(str(VALIDATED_RANGE_MONTHLY_FAMILIA), index=False)
print(f"Guardado df_monthly en: {VALIDATED_RANGE_MONTHLY_FAMILIA}")

# Vista previa

print(df_monthly.head())

Guardado df_montly en: ..\data\interim\validated_range_monthly_familia_20250630.parquet
        fecha   familia  base_imponible   total  is_easter  is_summer_peak
1  2023-01-31  AÑADIDOS           34.06   37.15        0.0             0.0
14 2023-02-28  AÑADIDOS           43.04   47.10        0.0             0.0
27 2023-03-31  AÑADIDOS           77.37   84.70        7.0             0.0
40 2023-04-30  AÑADIDOS          136.88  149.60       10.0             0.0
53 2023-05-31  AÑADIDOS           98.90  108.00        0.0             0.0


C:\Users\jordi\AppData\Local\Temp\ipykernel_9520\1963071782.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  meses = pd.date_range(df_daily['fecha'].min(), df_daily['fecha'].max(), freq='M')
C:\Users\jordi\AppData\Local\Temp\ipykernel_9520\1963071782.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample('M')
